# 여행 AI Agent

## 초기 설계
1. 사용자의 요구사항을 리스트로 만든다.
2. 각각의 리스트 항목에 대답한다. 우선 초기 모델이므로 LLM만을 이용하여 항목에 대한 답을 작성한다.
3. 작성한 답을 하나의 마크다운 파일에 작성한다.

### 초기 설계 노드 구성
1. extract_requirements
    * 입력: 사용자 프롬프트
    * 출력: 요구사항 리스트 항목들 (체크리스트 형태)
2. respond_to_checklist
    * 입력: 요구사항 리스트 항목들
    * 처리: 항목별로 LLM 호출하여 각 항목의 응답 생성
    * 출력: 응답이 포함된 리스트
3. review_responses
    * 입력: 응답이 포함된 리스트
    * 처리: LLM 의 답변을 마크다운 파일에 저장
    * 출력: 최종 결과물

## 수행 시간 개선
- 2번 작업을 비동기 + 배치로 전환

## 기능 개선
- Human In the Loop 추가
- 체크리스트 작성 후 사용자에게 확인받는 기능

## 한계
- 체크리스트에 대한 답을 얻을 때 LLM에만 의존.
- 진정한 Agent 라면 tool 을 붙여서 웹서치 등으로 답을 가져오는 편이..
- 답변들을 작성한 후 하나의 핸드북으로 만들기 위해 답변 취합 노드를 만드는 방법 이외에 유연한(혹은 철저한)설계로 해결할 수는 없을까?
  - Manus 는 이게 어떻게 되는거지 🥲


## 고민할 점
- OpenManus 실행해 봄
- 사용자의 의도를 이해하고 browser-use 까지 잘 활용하는지, 즉 적합한 tool 선택까지 잘 하는지 확인하고 싶었음
  - '5월 1일에 인천에서 도쿄로 향하는 항공편 정보를 제공해 줘' 라는 질문에 Skyscanner 를 가상 크롬 브라우저로 열어서 검색 시도
  - 인천 -> 도쿄 도시 자정은 되었으나 날짜 지정이 되지 않았음
  - 그럼에도 사용자의 요구를 이해하고, 알맞은 웹사이트인 스카이스캐너를 열고, 도시 정보 입력까지 잘 수행하였음.
  - 2회 이상 시도 시 휴먼 인증을 요구받아서 막혔음..
  - Agent 를 구현하는 데 고려할 점은? 설계? 모듈화??

In [ ]:
from dotenv import load_dotenv

# .env 파일의 환경변수를 기존 값과 상관없이 덮어쓰기
load_dotenv(override=True)

In [2]:
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain.schema import BaseOutputParser
from typing import TypedDict, List
import json

# 1. 상태 정의
class TripPlanningState(TypedDict):
    user_input: str
    user_info: str
    checklist: List[str]
    checklist_with_responses: List[dict]
    final_output: str

In [3]:
# 2. LLM 초기화
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat:free",
    # model_name="meta-llama/llama-3.3-70b-instruct:free",
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
    # streaming=True
    streaming=False
)

In [4]:
# 3. 노드 정의 : 기본 여행정보 추출
def get_travel_info(state: TripPlanningState) -> TripPlanningState:
    prompt = f"""
    당신은 여행 전문가입니다. 사용자의 여행 요구사항을 읽고, 여행 계획 체크리스트를 최대 10개 항목으로 요약해 주세요.  
    각 항목은 반드시 아라비아 숫자(1~10)로 시작해야 하며, 핵심 정보만 간단하고 명확하게 정리해야 합니다.  
    다음 요소를 반드시 포함하세요:  
    - 여행 일정  
    - 출발지 및 목적지  
    - 예산 범위  
    - 관심 있는 활동 및 테마  
    - 특별한 요청이나 계획 (예: 프로포즈 장소 등)

    사용자의 감정이나 상황을 과하게 해석하지 말고, 객관적인 계획 요소만 추출하세요.

    사용자 요구사항: {state['user_input']}
    """
    checklist_md = llm.invoke(prompt).content
    checklist_lines = checklist_md.strip().splitlines()
    # checklist_items = [line for line in checklist_lines if line.strip().startswith("- ") or line.strip().startswith("* ") or line.strip().startswith("1.")]

    # 상위 10개까지만 잘라내기
    checklist_items = checklist_lines[:10]

    return {**state, "checklist": checklist_items}

In [11]:
# 3. 노드 정의
def create_travel_checklist(state: TripPlanningState) -> TripPlanningState:
    prompt = f"""
    당신은 여행 전문가입니다. 사용자의 여행 요구사항을 읽고, 여행 계획 체크리스트를 최대 10개 항목으로 요약해 주세요.  
    각 항목은 반드시 아라비아 숫자(1~10)로 시작해야 하며, 핵심 정보만 간단하고 명확하게 정리해야 합니다.  
    다음 요소를 반드시 포함하세요:  
    - 여행 일정  
    - 출발지 및 목적지  
    - 예산 범위  
    - 관심 있는 활동 및 테마  
    - 특별한 요청이나 계획 (예: 프로포즈 장소 등)

    사용자의 감정이나 상황을 과하게 해석하지 말고, 객관적인 계획 요소만 추출하세요.

    사용자 요구사항: {state['user_input']}
    """
    checklist_md = llm.invoke(prompt).content
    checklist_lines = checklist_md.strip().splitlines()

    # 상위 10개까지만 잘라내기
    checklist_items = checklist_lines[:10]

    return {**state, "checklist": checklist_items}

In [6]:
import asyncio
from typing import List


async def async_respond_to_checklist_in_batches(state: TripPlanningState, batch_size: int = 3) -> TripPlanningState:
    async def get_response(item):
        if not item.strip():
            return {"item": item, "response": ""}
        return {
            "item": item, 
            "response": (await llm.ainvoke(f"'{item}' 이 항목에 대해 자세히 조사해서 설명해 줘. 여행 전문가처럼 답변해 줘.")).content
        }
    
    checklist = state["checklist"]
    all_responses = []
    
    # 체크리스트를 배치 크기로 나누어 처리
    for i in range(0, len(checklist), batch_size):
        batch = checklist[i:i + batch_size]
        print(f"📋 배치 {i//batch_size + 1} 처리 중... ({len(batch)}개 항목)")
        
        # 현재 배치의 모든 항목을 병렬로 처리
        batch_responses = await asyncio.gather(*[get_response(item) for item in batch])
        all_responses.extend(batch_responses)
        
        # 다음 배치 처리 전 잠시 대기 (API 레이트 리밋 방지)
        if i + batch_size < len(checklist):
            print(f"⏳ 다음 배치 처리를 위해 2초 대기...")
            await asyncio.sleep(2)
    
    print(f"✅ 총 {len(all_responses)}개 항목 처리 완료!")
    
    return {
        **state, 
        "checklist_with_responses": all_responses
    }

In [7]:
from langgraph.graph import StateGraph, END
from typing import Callable

def review_responses(state: TripPlanningState) -> TripPlanningState:
    print("\n🔍 각 항목에 대한 응답입니다.")
    
    # travel.md 파일을 생성하고, 내용을 작성
    with open("travel.md", "w", encoding="utf-8") as file:
        file.write("# 여행 체크리스트 및 응답\n\n")
        for r in state["checklist_with_responses"]:
            r['response'] = llm.invoke(f"'다음 내용 중 5월 15일부터 5월 21일까지 일본을 여행하는 것과 관련 있는 내용만 추출해 줘. {r['response']}'").content
            file.write(f"### {r['item']}\n{r['response']}\n\n")  # 마크다운 형식으로 저장

    print("\n✔️ travel.md 파일이 생성되었습니다.")
    return state

In [8]:
# 2. LangGraph 구성 (HITL 포함)
builder = StateGraph(TripPlanningState)
builder.add_node("create_travel_checklist", create_travel_checklist)
builder.add_node("respond_to_checklist", async_respond_to_checklist_in_batches)
builder.add_node("review_responses", review_responses)

builder.set_entry_point("create_travel_checklist")
builder.add_edge("create_travel_checklist", "respond_to_checklist")
builder.add_edge("respond_to_checklist", "review_responses")
builder.add_edge("review_responses", END)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)

In [ ]:
# 비동기 함수 실행용 main
async def main():
    result = await graph.ainvoke({
        "user_input": "저는 8월 15-23일 시애틀에서 출발하는 7일간의 일본 여행 일정이 필요하고, 저와 약혼자를 위해 2,500-5,000달러의 예산이 필요합니다. 저희는 역사적 유적지, 숨겨진 보석, 일본 문화(검도, 다도, 선 명상)를 좋아합니다. 저희는 나라의 사슴을 보고 도보로 도시를 탐험하고 싶습니다. 저는 이 여행 중에 프로포즈할 계획이며 특별한 장소 추천이 필요합니다."
    })
    print(result)

# 직접 await 호출 (Jupyter 환경에서 사용)
await main()